This is just a note for my learning of transformer, referred to articles and codes from

- https://www.tensorflow.org/tutorials/text/transformer
- https://github.com/MorvanZhou/NLP-Tutorials/blob/master/transformer.py
- https://zhuanlan.zhihu.com/p/347904940
- https://datawhalechina.github.io/learn-nlp-with-transformers

All credits go to the above authors.

The notes / comments are my understanding at this moment, please correct me if they are wrong.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Transfomer model

In [ ]:
class transformer(keras.Model):
    def __init__(self):
       pass

## Input embedding
Transfer inputs to vector with positional info.

input layer -> hidden layer -> output layer.
> - Mr.Wagaru: Do you understance what you are talking about?
> - me: After position embedding, you can get the weight of a word since the location of each word in one-hot encoder is deifferent ...
> - Mr.Wagaru: Do you really understand it? really understand?? really???
> - me: はい、はい、引き続き頑張ります。

If input is [batch_size, sequence_length], after embedding, we get a tensor of [batch_size, sequence_length, embedding_dimension].
* The batch size is the number of batch to transformer, in human's language, the number of sentences.
* The sequence length is the size of samples in one inputting, that is, the words number in one sentence. Since not all sentences have same length, this value should be the possible longest length.
* The embedding dimention is decided by Word2Vec. For transformer make it 512.

In [ ]:
class Embedding(keras.layers.Layer):
    def __init__(
        self, 
        embed_dimension: int, 
        batch_size     : int,
        sequence_length: int
        ):
        """
        @arg
        embed_dimension:
            The dimmention of embedding, or feature number.
        batch_size:
            If the sentences to be trained is 10, the batch_size is 10.
        sequence_length:
            The words size in one sentence. 
            If there has 8 words in one sentence, the value is 8. 
            However, since it's impossible for all sentences have same size, 
            this value should be set to a size of the possibe longest sentence.
        """
        super(Embedding, self).__init__()
        self.e_dim  = embed_dimension
        self.b_size = batch_size
        self.s_size = sequence_length
        self.v_size = vocabulary_size

        

        # [sequence_size, embed_dimension]
        pos = np.arange(s_size)[:, None]
        # positional_encoding = pos / np.power(10000, 2. * np.arange(e_dim)[None, :] / e_dim)
        

        positional_encoding = np.array(
            [
             [p / np.power(10000, 2 * i for i in np.arange(e_dim)[None, :] / e_dim)] if p !=0
             else ne.zeros(e_dim)   # padding zeros
             for p in pos
            ]
        )
        
        # The formula for calculating the positional encoding is as follows:
        # PE(pos, 2i) = sin(pos/10000^(2i/embed_dimension))
        # PE(pos, 2i+1) = cos(pos/10000^(2i/embed_dimension))
        # and the formula are not applied to the padding (the first one)

        positional_encoding[:, 0::2] = np.sin(positional_encoding[1:, 0::2])
        positional_encoding[:, 1::2] = np.cos(positional_encoding[1:, 1::2])
        
        self.positional_encoding = tf.cast(positional_encoding, dtype=tf.float32)
        
        # keep the embedding layer object
        self.embedding = keras.layers.Embedding(
            input_dim, # vocabulary size. Since encoder and decoder may have different v_size, left it to be decided in invoker, 
            output_dim=embed_dimension,
            embeddings_initializer=tf.initializers.RandomNormal(0., 0.01),
        )

    def __call__(self, vocabulary_size: int):
        """
        vocabulary_size:
            The vocabulary table dimention. 
            Say, if we have a vocabulary table which containing 1000 words, 
            this value is 1000.
            NOTE: the vocabulary sizes of decoder input and encoder target
                  are not necessarily to be same.
        """
        return self.embedding(vocabulary_size) + self.positional_encoding;

## Encoder

In [ ]:
class encoder(keras.layers.Layer):
    pass


## Decoder

In [ ]:
class decoder():
    pass